In [ ]:
#  Libraries

In [3]:
import os, csv, re, glob, math, collections, sys, codecs
import xml.etree.ElementTree as ET
import pandas as pd
from matplotlib import pylab as plt
import matplotlib.pyplot as plot
import seaborn as sns
import numpy as np
from numpy import *
from scipy import signal
from scipy.signal import find_peaks
from scipy import stats
from io import BytesIO
from colorama import *

#   Profile extraction from XML file - Scri(P)t

In [ ]:
print(Fore.BLUE + '####################################\n# Welcome to the SalivaPrint tool. #\n#----------------------------------#\n#        Type [y] to start.        #\n####################################')
choice=''
choice = input()
if choice=='y':
    print(Fore.RED + '-> Data Extraction from XML file.<-\n')
else:
    sys.exit(Fore.RED + 'Stop')
#-Next step is add a condition to input the Sample ID and confirm if is the of XML file to continue-#

* Createa csv data from xml source

In [ ]:
full_xml_path = os.path.abspath(os.path.join(".", "test.xml"))
full_csv_path = os.path.abspath(os.path.join(".", "Samples.csv"))


tree = ET.parse(full_xml_path)
root = tree.getroot()

file_header = ['Sample']
file_header_vals = []
sample_well_info = []


*   Set File Header

In [ ]:
for i in range(466, 869):
    file_header.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))
    file_header_vals.append(str(round((0.00042006 * (i * i) - 0.28216 * i + 49.225), 1)))

*   get all well id's and sample names from file

In [ ]:
for sample in root.findall('Results'):
    # write sample name
    well_id = sample.find('WellId').text

    if well_id != '0':
        sample_name = sample.find('SampleName').text
        sample_well_info.append({'Sample': sample_name, 'well_id': well_id})

*   create file with headers and data

In [ ]:
with open(full_csv_path, 'w', newline='') as outcsv:
    writer = csv.DictWriter(outcsv, fieldnames = file_header)
    writer.writeheader()
    xmlstr = ET.tostring(root, encoding='utf8', method='xml').decode("utf-8")

    for elem in sample_well_info:
        
        idx = 0
        row = {'Sample': elem['Sample']}
        regex = '<RawDataPoints><WellId>%s<\/WellId><Signal>(.*?)<\/Signal><\/RawDataPoints>'% elem['well_id']
        raw_data_points = re.findall(regex, xmlstr)
        
        # write data for each column
        for count, raw_data_point in enumerate(raw_data_points, start=0):
            if 869 >= count >= 467:
                row[file_header_vals[idx]] = raw_data_point
                idx+=1
            
            if count > 869:
                break
        
        writer.writerow(row)

*   plot profiles

In [ ]:
df = pd.read_csv('Samples.csv', index_col='Sample')
df.head(2)

df2 = df.T
df2.to_csv('output.csv')
df2.head(2)

ax = df2.plot()
ax.set_xlabel('Molecular Weight (kDa)')
ax.set_ylabel('Fluorescence')

In [ ]:
#Do you want to plot
print(Fore.GREEN + '#########################################\n#    Do you want to plot the profile?   #\n#---------------------------------------#\n#  Type [y] to plot or [n] to continue. #\n#########################################')
choice = ''
choice = input()
if choice == 'y':
    print(Fore.RED + 'Line Graph Plot.')
    plt.show()
elif choice == 'n':
    print(Fore.BLUE + 'Continue the profile analysis.')
else:
    sys.exit('Stop')


#  Signature finding: application of statistical test (i.e. Kruskal-Wallis) to find the Molecular Weight where groups are more distinct

In [5]:
samples = pd.read_csv("Samples.csv",header=0) #import XML extractor output
metadata = pd.read_csv("good_dataset.csv",header=0) #Get sample metadata

In [ ]:
*   Decision maker - Choose the variables on analysis: Gender; Age group; ... (not finished yet!!!!)

In [ ]:
# print(Fore.RED + '#########################################\n#    Wich groupdo you want to analyse?   #\n#---------------------------------------#\n#  Type [Gender] or [Age Group] to perform kruskal-wallis. Otherwise continue. #\n#########################################')
# choice = ''
# choice = input()
# if choice == 'Gender':
#     print(Fore.RED + 'Gender Analysis')
#     plt.show()
# elif choice == 'Age Group':
#     print(Fore.BLUE + 'Age Group Analysis.')
# else:
#     sys.exit('Continue')

In [6]:
stat = metadata[['Sample','Gender']].merge(samples, how ='inner', on='Sample') #Merging Metadata with Prints
stat1 = stat.drop(columns='Sample')
stat1.to_csv('kruskal.csv')
stat1.head(1)

,Gender,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.9,...,116.8,117.2,117.7,118.1,118.6,119.0,119.5,119.9,120.3,120.8
0,Male,-66.899189,-57.95638,-49.865632,-42.760716,-36.76062,-31.847161,-27.881861,-24.728603,-22.407828,...,-0.797122,-0.966253,-1.07896,-1.069842,-1.058991,-1.039013,-1.066419,-1.23437,-1.381473,-1.498213


* Levene test

In [7]:
grouped = stat1.groupby('Gender')
grouped.head()

,Gender,9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.9,...,116.8,117.2,117.7,118.1,118.6,119.0,119.5,119.9,120.3,120.8
0,Male,-66.899189,-57.956380,-49.865632,-42.760716,-36.760620,-31.847161,-27.881861,-24.728603,-22.407828,...,-0.797122,-0.966253,-1.078960,-1.069842,-1.058991,-1.039013,-1.066419,-1.234370,-1.381473,-1.498213
1,Male,-69.882155,-60.823732,-52.619868,-45.468589,-39.379809,-34.363317,-30.345212,-27.099656,-24.496454,...,-0.413701,-0.390115,-0.446808,-0.535110,-0.626226,-0.860594,-1.046076,-1.174095,-1.284146,-1.544778
2,Female,-101.624743,-103.154567,-101.005651,-97.943508,-93.704260,-87.971012,-81.504261,-75.177119,-69.458292,...,0.638402,0.096138,-0.086788,-0.017538,0.033578,-0.050845,-0.184136,-0.296660,-0.464255,-0.640865
3,Female,-112.338401,-114.021760,-107.965437,-100.954144,-94.979890,-88.949258,-82.420227,-75.941756,-70.016267,...,1.661894,1.114958,0.608922,0.266198,0.148713,0.154016,0.132912,0.052730,-0.066392,-0.204968
4,Female,64.030962,14.903387,-21.367195,-45.827966,-60.419500,-67.417181,-68.962492,-67.116328,-63.514611,...,-0.619201,-0.463723,-0.416500,-0.584152,-0.902654,-1.154787,-1.241280,-1.299298,-1.467843,-1.809336
5,Female,60.843186,8.342853,-25.010124,-44.936784,-56.969658,-64.730808,-69.109687,-70.392210,-69.159204,...,-0.612183,-0.655345,-0.815721,-0.978116,-1.071411,-1.115840,-1.181168,-1.339735,-1.609237,-1.972195
6,Female,68.918498,11.947969,-19.189377,-33.686199,-39.014039,-41.236305,-42.013056,-41.300465,-39.335822,...,-0.481412,-0.331217,-0.211620,-0.180858,-0.265307,-0.424616,-0.570542,-0.715060,-0.945225,-1.227621
16,Male,-54.015308,-65.326773,-64.685859,-59.112713,-51.891025,-44.463955,-37.754285,-32.256899,-27.943486,...,-0.257578,-0.382063,-0.660176,-0.893811,-0.920388,-0.807778,-0.843335,-1.069970,-1.403615,-1.761690
17,Male,-36.074493,-51.368412,-55.463124,-53.807609,-49.615077,-44.110845,-38.377580,-33.187815,-28.854714,...,-0.260127,-0.332794,-0.525458,-0.643734,-0.673905,-0.669570,-0.645519,-0.679419,-0.880167,-1.286263
22,Male,23.242946,18.832955,16.155902,13.015589,10.062717,7.639099,5.695924,4.212974,3.096781,...,-0.572671,-0.478495,-0.413014,-0.422997,-0.494617,-0.589010,-0.696136,-0.841071,-0.975210,-1.157331


In [8]:
from scipy import stats
grouped = stat1.groupby('Gender')
for name, group in grouped:
    g1 = stat1[(stat1['Gender'] == 'Female')]
    g2 = stat1[(stat1['Gender'] == 'Male')]
    # print(name)
    # print(group)
mw = list(group.columns)
mw.remove('Gender') #get the mw to perform loop

In [1]:
L = []
for e in mw:
    levene = stats.levene(g1[e], g2[e])
    # print(levene)
    L.append(levene)
df = pd.DataFrame(L)
df['MW'] = mw
df.head()

NameError: name 'mw' is not defined

In [ ]:
#box plots to levene test

In [ ]:
plot.figure(1)
plot.subplot(2, 1, 1)
for name, value in g1.items():
    plot.plot(value, 'b')
for name, value in g2.items():
    plot.plot(value, 'r')
plt.show()

In [12]:
#kruskal
K = []
for e in mw:
    kruskal = stats.kruskal(g1[e], g2[e])
    # print(levene)
    K.append(kruskal)
df = pd.DataFrame(K)
df['MW'] = mw
print(df)

statistic    pvalue     MW
0     0.024390  0.875896    9.0
1     0.878049  0.348737    9.1
2     1.264390  0.260822    9.2
3     1.195122  0.274299    9.3
4     1.640000  0.200325    9.4
..         ...       ...    ...
398   0.711220  0.399039  119.0
399   0.609756  0.434880  119.5
400   0.937561  0.332906  119.9
401   1.640000  0.200325  120.3
402   1.195122  0.274299  120.8

[403 rows x 3 columns]


In [13]:
from pingouin import kruskal, read_dataset
df = read_dataset('anova')
df.head(1)
kruskal(data=df, dv='Pain threshold', between='Hair color')

ModuleNotFoundError: No module named 'pingouin'

In [14]:
# kruskall wallis
from scipy import stats
kruskal = []
pvalue = []
for e in mw:
    kruskal, pvalue = stats.kruskal(g1,g2)
    kruskal_values.append(kruskal)
    p_value.append(pvalue)

ModuleNotFoundError: No module named 'unittest'

In [ ]:
print(kruskal)

*   plot the means

In [ ]:
groupby = stat.groupby('Gender').mean().T
groupby.head()

In [ ]:
kkw = groupby.plot()
kkw.set_xlabel('Molecular Weight (kDa)')
kkw.set_ylabel('Fluorescence')

In [ ]:
groupby = stat.groupby('Gender').describe()
groupby.head()

In [ ]:
female = stat[stat['Gender'] == 'Female'] #Gender Groups
g1 = female.drop(columns='Gender').T
male = stat[stat['Gender'] == 'Male']
g2 = male.drop(columns='Gender').T
g1.head(1)

In [ ]:
values = samples.loc[:,'9.0':'120.8'] #Normalise values 
soma_total = np.sum(values)
normalization = np.true_divide(values, 1000 / soma_total)
normalization.head(1)

In [ ]:
def Gender(stat):
    if 'Gender' == 'Female':
        plot.plot(values, 'r')
    elif 'Gender' == 'Male':
        plot.plot(values, 'b')

In [ ]:
plot.figure(1)
plot.subplot(2, 1, 1)
# g1.plot()
plot.plot(g1, 'r')
plot.plot(g2, 'b')

### Statistical analysis

* data import

In [ ]:
k = metadata[['Sample','Gender']].merge(samples, how ='inner', on='Sample')
k.head(2)

*   Test Dataset

In [ ]:
g1.describe
print('g2: mean=%.3f stdv=%.3f' % (mean(g2), std(g2)))

In [ ]:

female = k[k['Gender'] == 'Female'] #Gender Groups
g1 = female.drop(columns=['Gender', 'Sample']).T
male = k[k['Gender'] == 'Male']
g2 = male.drop(columns=['Gender','Sample']).T
k.head(1)

In [ ]:
g2.head(1)

*   Kruskal-Wallis hypothesis test

In [ ]:
pg.kruskal(data=p1, dv='9.0' ,between='Gender')

In [ ]:
kruskal_values = []
p_value = []

In [ ]:
stats.kruskal(p1)

In [ ]:
kruskal, pvalue = stats.kruskal(g1, g2)


In [ ]:
kruskal_values.append(kruskal)
p_value.append(pvalue)

In [ ]:
kruskal_values = np.array(kruskal_values)
plot.plot(kruskal_values, 'g')
plot.plot(p_value, 'y')

In [ ]:
plot.subplot(2, 1, 2)
plot.plot(kruskal_values, 'g')
plot.plot(p_value, 'y')

In [ ]:
peaks, _ = find_peaks(kruskal_values, height=0, prominence=10)
plot.plot(peaks, kruskal_values[peaks], "x")
plot.show()

plot.figure(2)

In [ ]:
axisg1 = np.zeros(len(g1))
axisg2 = np.ones(len(g2))

In [ ]:
plot.plot(axisg1, g1, 'ob')
plot.plot(axisg2, g2, 'or')
plot.show()


In [ ]:
# Importing the required libraries
import pandas as pd

In [ ]:
# Two different methods of conducting a t-test
import researchpy as rp
from scipy import stats

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master//Iris_Data.csv")
df.head()

In [ ]:
setosa = df[(df['species'] == 'Iris-setosa')]
setosa.reset_index(inplace= True)
setosa.head()

In [ ]:
versicolor = df[(df['species'] == 'Iris-versicolor')]
versicolor.reset_index(inplace= True)

In [ ]:
stats.levene(setosa['sepal_width'], versicolor['sepal_width'])

In [ ]:
diff = setosa['sepal_width'] - versicolor['sepal_width']

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

stats.probplot(diff, plot= plt)
plt.title('Sepal Width P-P Plot') 
plt.show()

In [ ]:
diff.plot(kind= "hist", title= "Sepal Width Residuals")
plt.xlabel("Length (cm)")
plt.show()

In [ ]:
stats.shapiro(diff)

In [ ]:
descriptives, results = rp.ttest(setosa['sepal_width'], versicolor['sepal_width'])

descriptives

In [ ]:
results

In [ ]:
from pingouin import kruskal, read_dataset
df = read_dataset('anova')
kruskal(data=df, dv='Pain threshold', between='Hair color')